# Data Preparation for ClassicVC

Lyodos 著

Version 1.0.0 (2024-07-14)

## データセット

ClassicVC の訓練には多数（数千～数万名以上）の話者による、なるべく高音質かつ、他の話者の声が混ざらず、BGM がなく、
部屋鳴りの極力少ない発話データが必要である。

連続な潜在空間で話者スタイルを表現し、かつ未知の話者に対しても妥当な声質変換を行うことを、本モデルの目標とする。
そして可能な限り、人類の表現型の大部分をカバーする声質が均等に得られることが望ましい。
そのためには、従来の VC の研究で用いられてきた VCTK 等のデータセットでは話者数が不足している。

一方、TTS（text to speech）タスクとは異なり、テキスト書き起こしは不要である。
したがって話者認識等のタスク用に作成された、
テキストを含まない（ゆえに数を増やしやすい）データセットを使用できる。

以下で入手して訓練に用いる公開データセット（LibriSpeech, Samrómur Children 21.09,  VoxCeleb 1 and 2）は、
いったん音質には妥協してでも多様なバックグラウンドを持つ、多数の話者の声を収集することを目指して選定されたものである。
いずれも音声分野における大規模研究プロジェクトの成果物として整備されたオープンデータであり、
Creative Commons ライセンスに従って利用可能だ。

なおこれらのデータセットに、日本語の発話はほぼ含まれていない。
しかし主要なソース言語である英語に含まれる音素の種類が日本語よりも多いことから、
結果的には日本語の声質変換も問題なく行えることが判明している。


> 注意：以下の方法でデータセットを準備するためは、数百 GB 単位の SSD の空き容量が必要。
> さらに VC モデルの訓練ログの吐き出しまで考慮すると、1 TB 程度の容量は確保したい。



----

## 1. LibriSpeech ASR corpus 

Summary: Large-scale (1000 hours) corpus of read English speech

名前の似ている LibriTTS や LibriTTS-R もあるが、LibriSpeech を使う。

音声ファイルは 16 kHz, 16 bit の flac 形式である。
なお 44.1 kHz のオリジナル mp3 ファイルもダウンロードできるが、1 回分の朗読音声が 1 つのファイルになっており分割が必要なので、
今回は諦めた。
これをスクリプトで（ゼロクロスポイントを見つけて）5 秒程度ずつのクリップに分割すれば、将来的には decoder をより高音質で訓練できるだろう。
余力のある人は挑戦してほしい。

論文

* V. Panayotov, G. Chen, D. Povey and S. Khudanpur, "Librispeech: An ASR corpus based on public domain audio books," 2015 IEEE International Conference on Acoustics, Speech and Signal Processing (ICASSP), South Brisbane, QLD, Australia, 2015, pp. 5206-5210, doi: 10.1109/ICASSP.2015.7178964.

* https://ieeexplore.ieee.org/document/7178964

OpenSLR にある配布ページ

* https://openslr.org/12/ 

配布ファイルがたくさんあるが、以下の 3 つをダウンロードする。

* train-clean-100.tar.gz [6.3G]   (training set of 100 hours "clean" speech )

* train-clean-360.tar.gz [23G]   (training set of 360 hours "clean" speech ) 

* train-other-500.tar.gz [30G]   (training set of 500 hours "other" speech )   

これらのサブセットは大雑把に音声品質がクリーンな順でグループ分けされている。
今回の訓練では、とりあえず話者数が多い方がよかろうと考えて全部使っている。

### License

* Creative Commons - Attribution 4.0 International (CC BY 4.0)

### ダウンロードしたファイルの配置

Jupyter Notebook からアクセス可能な、マシン上のディレクトリ上に `dataset` というフォルダを作成する。
ここがデータセットたちをまとめるルートフォルダになる。
その下に `librispeech` というサブフォルダを作って、ダウンロードしたファイルを解凍する。

たとえば Linux において `/home/lyodos/study/dataset` をルートに定めたとき（ユーザー各位は読み替えてほしい）、

* `/home/lyodos/study/dataset/librispeech/train-clean-100`

* `/home/lyodos/study/dataset/librispeech/train-clean-360`

* `/home/lyodos/study/dataset/librispeech/train-other-500`

という 3 つの解凍先フォルダができるようにする（もし解凍されたフォルダの名前が異なったら、修正する）。

これら 3 つのフォルダ各々について、tar.gz を解凍して現れる多数の数字名称のフォルダが、直下に配置されるようにする。
これらのフォルダは「話者」に対応しており、`train-clean-100` では 251 個（話者）、
`train-clean-360` では 921 個、`train-other-500` では 1166 個ほど存在する。

* 各話者フォルダの下にはさらに、1 つないし複数のフォルダが存在する。これは各話者が朗読したオーディオブックの数だけ存在する。

* 各オーディオブックフォルダの下に、朗読をさらに数秒ないし数十秒ごとに区切った「発話 utterance」の音声ファイルが入っている。

つまり Librispeech では、

```
データセットを全部まとめたルート/
各データセット/
各データセットのサブセット/
話者/
オーディオブック/
音声ファイル
``` 

という、フォルダの階層構造を作ることになる。





---

## 2. Samrómur Children 21.09

Summary: Samrómur Icelandic Speech from children (ages 4-17 years) approved for release in September 2021

音声ファイルの形式は 16 kHz, 16 bit linear PCM, 1 ch の flac ファイル。

公式

* https://repository.clarin.is/repository/xmlui/handle/20.500.12537/185

引用情報

* Mena, Carlos; et al., 2021, Samromur Children 21.09, CLARIN-IS, http://hdl.handle.net/20.500.12537/185. 

OpenSLR にある配布ページ

* https://openslr.org/117/ 3175 話者、ただし録音品質が低い or 発話クリップ数が少なすぎる話者（train 54話者、dev 3 話者、test 264 話者）をダウンロード後に手動で除外した


### License

* Creative Commons - Attribution 4.0 International (CC BY 4.0)

### ダウンロードしたファイルの配置

先ほどと同じルートフォルダに倣い、直下に `samr_child` というサブフォルダを作って、ダウンロードしたファイルを解凍する。
こちらは解凍すると、 data_train / data_dev / data_test というサブセットに分かれ、やはり下に話者ごとのフォルダが並ぶ（ちなみに dev が他のデータセットの val に相当する）。

* `/home/lyodos/study/dataset/samr_child/data_train`

* `/home/lyodos/study/dataset/samr_child/data_dev`

* `/home/lyodos/study/dataset/samr_child/data_test`

という 3 つの解凍先フォルダができる。サブフォルダは `data_train` では 2463 個（話者）、
`data_dev` では 31 個、`data_test` では 361 個ほど残す。

このデータセットについてはまとまった文章の朗読ではなくバラバラな発話を収録しているらしいので、
話者フォルダの下に直接、数秒ないし数十秒ごとに区切った「発話 utterance」の音声ファイルが入っている。

つまり Samrómur Children 21.09 では

```
データセットを全部まとめたルート/
各データセット/
各データセットのサブセット/
話者/
音声ファイル
``` 

という、フォルダの階層構造を作ることになる。




----

## 3. VoxCeleb 1 and 2

Visual Geometry Group, Department of Engineering Science, University of Oxford による公式サイト（2024 年現在、音声配布は停止中）

* https://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox1.html 1251 話者

* https://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox2.html 6112 話者

Multimodal AI Lab, KAIST によるミラー（こちらから音声をダウンロードできるが、登録が必要）

* https://mm.kaist.ac.kr/datasets/voxceleb/

最近は Hugging Face からも落とせるようになった

* https://huggingface.co/datasets/ProgramComputer/voxceleb/tree/main/vox2

論文

<ul>
    <li>
        <p><b>VoxCeleb: a large-scale speaker identification dataset</b><br>
        A. Nagrani*, J. S. Chung*, A. Zisserman<br>
        Interspeech, 2017<br>
        <a href="http://www.robots.ox.ac.uk/~vgg/publications/2017/Nagrani17/nagrani17.pdf">PDF</a></p>
    </li>
    <li>
        <p><b>VoxCeleb2: Deep Speaker Recognition</b><br>
        J. S. Chung*, A. Nagrani*, A. Zisserman<br>
        Interspeech, 2018<br>
        <a href="http://www.robots.ox.ac.uk/~vgg/publications/2018/Chung18a/chung18a.pdf">PDF</a></p>
    </li>
    <li>
        <p><b>VoxCeleb: Large-scale speaker verification in the wild</b><br>
        A. Nagrani*, J. S. Chung*, W. Xie, A. Zisserman<br>
        Computer Speech and Language, 2019<br>
        <a href="http://www.robots.ox.ac.uk/~vgg/publications/2019/Nagrani19/nagrani19.pdf">PDF</a></p>
    </li>
</ul>


音声ファイルは VoxCeleb 1 が 16 kHz, 16 bit linear PCM, 1 ch の wav ファイル。
**VoxCeleb 2 は m4a ファイルであり、後述するように PyTorch で円滑に取り扱うためには事前変換が必要となる**。

### License

公式サイトの記述によると、

* The VoxCeleb dataset is available to download for research purposes under a Creative Commons Attribution 4.0 International License. 

* The copyright remains with the original owners of the video.



### ダウンロードしたファイルの配置

先ほどと同じ `dataset` というフォルダの直下に `voxceleb1` および `voxceleb2` というサブフォルダを作って、
ダウンロードしたファイルを解凍する。

#### VoxCeleb1

VoxCeleb1 は vox1_dev_wav.zip (30.3 GB) と vox1_test_wav.zip (0.99 GB) という 2つの zip ファイルがあり、
それぞれ解凍すると下に話者ごとのフォルダが並ぶ。

* `/home/lyodos/study/dataset/VoxCeleb1/dev_wav`

* `/home/lyodos/study/dataset/VoxCeleb1/test_wav`

という 2 つの解凍先フォルダができるようにする。
サブフォルダは `dev_wav` では 1211 個（話者）、`test_wav` では 40 個ある。

このデータセットは YouTube からダウンロードした音声を、短時間の発話に切り出す形で用意されているらしい。
なので話者フォルダの下に「動画」を単位としたフォルダがあり、
さらにその中に数秒ないし数十秒ごとに区切った「発話 utterance」の音声ファイルが入っている。

つまり VoxCeleb 1 では、

```
データセットを全部まとめたルート/
各データセット/
各データセットのサブセット/
話者/
動画/
音声ファイル
``` 

という、フォルダの階層構造を作ることになる（これは VoxCeleb2 も同様）。

#### VoxCeleb2

問題は VoxCeleb2 だ。こいつのデータ容量は破滅的に大きい。
落とすファイルは dev set に相当する `vox2_aac.zip` だけだが 72.1 GB もある（Hugging Face の場合は 10 GB ずつ分割して落とせる）。

そして、解凍すると m4a 形式の音声が出てくる。
まあとりあえず、

* `/home/lyodos/study/dataset/VoxCeleb2/aac`

という解凍先フォルダに突っ込む。しかし m4a を Numpy や PyTorch で扱うのは少し面倒なので、次で書くように変換が必要である。


----


## メタデータの作成（と、VoxCeleb2 の m4a -> ogg 変換）

ここまでの作業でデータセットを配置したルートフォルダを設定したので、走査してメタデータを作る。

以下の一連のセルを実行することで、VoxCeleb2 の m4a 形式音声を ogg に変換しつつ、
その他のデータセットの音声クリップも一覧したリストを作成できる。

ただし変換に 1 日以上かかるので、データセットごとに分割して走らせてから、リストを繋いだ方がいいかもしれない。



In [ ]:

# 必要ならば https://github.com/kkroening/ffmpeg-python/tree/master をインストールする

#!pip install ffmpeg-python


In [ ]:

from pathlib import Path

min_len = 2.0 # 2.0 秒未満のクリップは捨てる
max_len = 40.0 # 40 秒以上のクリップも捨てる

# 以下は全てのデータセットが配置された共通のルートフォルダ。これから作るメタデータの保存先でもある。
DATASET_ROOT_PATH = Path("/home/lyodos/study/dataset") # このフォルダ名はユーザーの実情に合わせて書き変えること

# 以下はデータサブセットのフォルダ、つまり「話者フォルダの直上」までを指し示すように記載する
# 実際には VoxCeleb とそれ以外に分割して、後からリストを繋いだほうがいいと思う
SUBSET_DIRS = [
    DATASET_ROOT_PATH / "voxceleb2" / "aac", # 5994 話者。ここだけ m4a ファイルなので変換が必要になる
    DATASET_ROOT_PATH / "voxceleb1" / "dev-wav", # 1211 話者
    DATASET_ROOT_PATH / "voxceleb1" / "test-wav", # 40 話者
    DATASET_ROOT_PATH / "librispeech" / "train-clean-100", # 251 話者
    DATASET_ROOT_PATH / "librispeech" / "train-clean-360", # 921 話者
    DATASET_ROOT_PATH / "librispeech" / "train-other-500", # 1166 話者
    DATASET_ROOT_PATH / "samr_child" / "data_train", # 2517 → 2463 話者
    DATASET_ROOT_PATH / "samr_child" / "data_dev", # 34 → 31 話者
    DATASET_ROOT_PATH / "samr_child" / "data_test", # 625 → 361 話者
]

infos = [] # 話者フォルダを一覧して、メタデータを貯めていくリスト。まず空で初期化

# 転じてこちらは、VoxCeleb2 のフォーマットを ogg に変換した後の音声の置き場所を指定する（実際のフォルダは下で作る）
celeb2_wav_root = DATASET_ROOT_PATH / "voxceleb2" / "dev-ogg"

# もともと "voxceleb2" / "aac" にあった m4a ファイルを ogg に変換したものが "voxceleb2" / "dev-ogg" に新しく書き出されるので、
# ディスク容量に最低 100 GB 程度の余裕がないとヤバい。



以下が変換スクリプトだが、 
最初に / "samr_child" / "data_dev" のような小さなデータセットだけを SUBSET_DIRS に入れて、単独でテストすることをお勧めする。

> なお以下のスクリプトは for の中に try が入っているので、いったん動き出したら Jupyter のセルの停止ボタンでは停止できないことに注意。



In [ ]:
%%time

from fastprogress import master_bar, progress_bar
import time

import ffmpeg
import os
import json

import mgzip
import pickle

from librosa.util import find_files
from IPython.utils import io

mb = master_bar(SUBSET_DIRS)


# データセットフォルダごとの処理
for dataset_path in mb:
    dataset_name = str(dataset_path.parts[len(DATASET_ROOT_PATH.parts)])
    print(dataset_name)
    mb.write(f"Dataset {dataset_name}") # dataset_path はデータセット名に対応する
    
    # 内部に含まれる話者を一覧する。サブフォルダの階層の深さはデータセットごとに異なるので、麾下のディレクトリを総ざらえする
    spkr_paths = [x for x in dataset_path.iterdir() if x.is_dir()]
    # Path.iterdir() は、パスに含まれるファイルやディレクトリをイテレータ化する。
    pb = progress_bar(spkr_paths, parent = mb)
    
    # ここから話者フォルダごとの処理
    for i, spkr_path in enumerate(pb):
        start_b = time.time()
        spkr_name = spkr_path.name # Path オブジェクトの name は最下層のディレクトリ名を文字列で取り出す。
        audio_paths = find_files(spkr_path) # 各話者ディレクトリに含まれる音声ファイルをリスト
        # librosa.util.find_files() の機能なので、非音声が混入してもフィルタされる。
        uttrs = [] # 話者内の全発話メタデータ一覧
        
        # ここから発話ごとの処理
        for j, audio_path in enumerate(audio_paths):
            try:
                # probe とは ffmpeg でメディアファイルのメタデータを取る手段
                probe = ffmpeg.probe(str(audio_path))
                audio_path = Path(audio_path)
                # 長さが条件に当てはまるサンプルのみ処理
                if float(probe["format"]['duration']) >= min_len and float(probe["format"]['duration']) < max_len:
                    if probe["streams"][0]["codec_name"] == "aac":
                        # 音声のコーデックが aac の場合（VoxCeleb2 が該当）のみ、フォーマット変換が入る
                        path_parts = audio_path.parts # 音声ファイルのパスを階層で分割した文字列リストに変換
                        # 変換した VoxCeleb2 の音声クリップを配置するディレクトリは新造する
                        save_path = celeb2_wav_root / path_parts[-3] / path_parts[-2] # データセットルート / 話者 / 動画
                        save_path.mkdir(parents = True, exist_ok = True)
                        save_name = str(save_path) + "/" + Path(path_parts[-1]).stem + ".ogg"
                        # ffmpeg の処理フローで aac 音声をメモリにロードして ogg に変換
                        stream = ffmpeg.input(str(audio_path)).output(save_name, format = 'ogg', ac = 1, ar = '16k', q = 8.0)
                        # 指定したファイル名で ogg を書き出し
                        if not os.path.isfile(save_name):
                            with io.capture_output() as captured:
                                ffmpeg.run(stream, overwrite_output = True, quiet = True)
                    else:
                        # それ以外の形式（他のデータセット）では、音声ファイルの現在のパスだけ記録する
                        save_name = audio_path

                    # ファイルごとにメタデータ一覧の辞書を作る
                    uttrs.append({
                        "dataset": str(dataset_name),
                        "subset": str(audio_path.parts[len(DATASET_ROOT_PATH.parts)+1]),
                        "speaker": spkr_name,
                        "audio_path": str(audio_path),
                        "shape": (1, probe["streams"][0]['duration_ts']),
                        "sr": int(probe["streams"][0]["sample_rate"]),
                        "seconds": float(probe["streams"][0]["duration"]),
                    })
                else:
                    pass # 長さが短すぎたり長過ぎたりするサンプルをスキップ
            except:
                print(f"Failed: {dataset_name} ({i}, {j}) {audio_path}")
    
        mb.child.comment = 'Seconds per iteration = {:.3f}'.format( time.time() - start_b )
        # 話者に対応する infos キーに、 uuid で作った Mel ファイルのパスを追記する。
        infos.append(uttrs)
        
    # データセットごとに、そこまでの進捗を保存
    output_file = str(DATASET_ROOT_PATH / f"classic-vc-meta_until-{dataset_name}.pkl.gz")
    with mgzip.open(output_file, 'wb') as f:
        pickle.dump(infos, f)

# つまり、最終的に生成される infos はデータセットやサブセットの階層をすっとばして、
# 直下に話者の数だけ発話リスト uttrs をひたすら並べたフラットな構造になる。



上手くいったら保存


In [ ]:

import mgzip
import pickle
import os

# 保存先のファイルパス
output_file = str(DATASET_ROOT_PATH / "classic-vc-meta.pkl.gz")

# mgzipでgzip圧縮しながらpickle形式で保存する
with mgzip.open(output_file, 'wb') as f:
    pickle.dump(infos, f)

# 以下は圧縮せずに json に吐き出す。大きすぎるので最初のお試し時以外はお勧めしない
#with open(DATASET_ROOT_PATH / "classic-vc-meta.json", "w", encoding="utf-8") as f:
#    json.dump(infos, f, indent = 4, ensure_ascii = False) 

print(f"Saved data to {output_file}")



作成したメタデータの構造を見ておこうねぇ。

ただし滅茶苦茶に長いので、冒頭だけ表示させないと大変なことになる。



In [ ]:
print(len(infos)) # この長さが結合したデータセットに含まれる話者数を表す


In [ ]:
from rich.pretty import pprint

print(len(infos[0])) # この長さが最初の話者が持つ発話クリップ数を表す
pprint(infos[-1][0:3]) # とりあえず最初の 3 発話分のメタデータを表示してみる




**この方法で作ったメタデータには、パスの途中にユーザー名等の情報が入るので外部に公開するのは避けた方がよい。**

なお相対パスで作ると、WSL2 環境からホストの Windows のフォルダを参照するのが面倒になるため、絶対パスで作っている。

> ちなみに変換が無事完了した後の aac フォルダは、ディスク容量に不安があれば捨ててもよい。

